<a href="https://colab.research.google.com/github/jongkwangyun/quant/blob/main/%EB%8F%99%EC%A0%81%EC%9E%90%EC%82%B0%EB%B0%B0%EB%B6%84_BAA_agg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# 첫 설치 이후 설치 안함
!pip install yfinance

In [11]:
# 첫 설치 이후 설치 안함
!pip install pandas_market_calendars

In [12]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display, Markdown
import csv

# display에서 소수점 4자리까지 표현
pd.options.display.float_format = '{:.4f}'.format

# display시 DataFrame 모든 열 표시
pd.set_option('display.max_columns', None)

# 데이터 가져올 기간을 지정합니다. (오늘로부터 12개월 전)
today = datetime.today().date()
start_date = today - timedelta(days=375)

# 전략명
str_strategy = 'BAA_agg '

# 전략에 사용할 ETF 명 리스트에 담기
canary_etf = ['SPY', 'VEA', 'VWO', 'BND']
aggressive_etf = ['QQQ', 'VWO', 'VEA', 'BND']
defensive_etf = ['BIL', 'IEF', 'TLT', 'LQD', 'TIP', 'BND', 'DBC']

# 자산군 리스트 더하기, 중복 제거
etf_order = list(set(canary_etf + aggressive_etf + defensive_etf))

# ETF 가격 추출
df_etf = yf.download(etf_order, start=start_date, end=today)['Adj Close']\
.sort_index(ascending=False)[etf_order].rename_axis(None, axis=0)

display(df_etf)

[*********************100%%**********************]  11 of 11 completed


,TIP,SPY,BND,QQQ,VEA,LQD,IEF,BIL,DBC,TLT,VWO
2023-08-24,105.4500,436.8900,71.0900,361.2200,44.5800,104.9000,93.7200,91.7500,24.1700,94.9100,40.2600
2023-08-23,105.9300,443.0300,71.2600,369.1100,45.1500,105.2600,94.0700,91.7200,24.1500,95.5400,40.4300
2023-08-22,104.9800,438.1500,70.6200,363.3800,44.6700,103.8000,93.0600,91.7000,24.1000,93.2300,39.8400
2023-08-21,104.8000,439.3400,70.5300,363.9000,44.7700,103.4800,93.0200,91.6900,24.0900,92.5200,39.9100
2023-08-18,105.0800,436.5000,70.8700,358.1300,44.6300,104.1000,93.6000,91.6800,24.1500,93.7700,39.8200
...,...,...,...,...,...,...,...,...,...,...,...
2022-08-19,111.6793,415.3573,73.3639,320.5744,41.1656,107.3997,99.8817,88.0716,25.8042,109.7054,40.0994
2022-08-18,111.8728,421.0149,73.8304,326.9490,41.7882,108.6893,100.7491,88.0716,25.8141,111.5009,40.5529
2022-08-17,111.3020,419.7949,73.6944,326.1646,41.9244,108.4102,100.5541,88.0330,25.4265,111.3359,40.8134
2022-08-16,111.6406,422.7958,74.1317,329.9277,42.2843,109.4399,101.2364,88.0330,25.3271,112.5102,40.9099


In [13]:
# csv 경로 설정
output_path =  '/content/drive/MyDrive/DynamicAssetAllocation/'

# 오늘 날짜를 가져와서 파일명에 추가
today_date = datetime.now().strftime('%y%m%d')
filename = f'{today_date} BAA_agg.xlsx'

# 오늘 날짜 string에 저장
str_today = today.strftime('### %Y-%m-%d ')

# 월평균영업일수. average Monthly Business Day
MBD = 21

# 인덱스 만들기
df_momentum_index = ['today/' + str(df_etf.index[0].date()), '1mo_ago/' + str(df_etf.index[MBD].date()), \
                     '3mo_ago/' + str(df_etf.index[MBD*3].date()), '6mo_ago/' + str(df_etf.index[MBD*6].date()), \
                     '12mo_ago/' + str(df_etf.index[MBD*12].date())]


## DataFrame 만들기: 1-3-6-12 개월 etf 가격
price_today = df_etf.iloc[0:1]
price_1mo = df_etf.shift(-MBD).iloc[0:1]
price_3mo = df_etf.shift(-MBD*3).iloc[0:1]
price_6mo = df_etf.shift(-MBD*6).iloc[0:1]
price_12mo = df_etf.shift(-MBD*12).iloc[0:1]

df_price = pd.concat([price_today, price_1mo, price_3mo, price_6mo, price_12mo])
df_price.index = df_momentum_index

str_etf_price = str_today + str_strategy + "### ETF 현재-1-3-6-12 개월 가격"
display(Markdown(str_etf_price))
display(df_price)

# csv에 제목 및 DataFrame 추가. 드라이브 마운트 후 해야함
with open(output_path + filename, 'w') as f:
  f.write(f'{str_etf_price}\n')
  df_price.to_csv(f, index=True)

### 2023-08-25 BAA_agg ### ETF 현재-1-3-6-12 개월 가격

,TIP,SPY,BND,QQQ,VEA,LQD,IEF,BIL,DBC,TLT,VWO
today/2023-08-24,105.4500,436.8900,71.0900,361.2200,44.5800,104.9000,93.7200,91.7500,24.1700,94.9100,40.2600
1mo_ago/2023-07-26,107.2883,455.5100,72.4664,377.5200,47.4400,107.7999,96.1164,91.3363,24.4900,100.9916,42.4700
3mo_ago/2023-05-24,107.0516,409.5686,71.9391,331.1957,44.7268,105.3223,96.6515,90.5284,22.8300,99.7179,39.0574
6mo_ago/2023-02-23,105.5593,397.6595,71.2561,295.9552,44.1249,104.5472,94.5921,89.5846,23.7500,100.7140,39.9833
12mo_ago/2022-08-23,111.6503,405.7246,73.0044,311.8764,40.5820,106.7934,99.1994,88.0619,26.4602,108.7350,40.0704


In [14]:
## DataFrame 만들기: 수익률
ror_1mo = df_etf.pct_change(-MBD).iloc[0:1]*100    # 1개월 수익률
ror_3mo = df_etf.pct_change(-MBD*3).iloc[0:1]*100    # 3개월 수익률
ror_6mo = df_etf.pct_change(-MBD*6).iloc[0:1]*100   # 6개월 수익률
ror_12mo = df_etf.pct_change(-MBD*12).iloc[0:1]*100  # 12개월 수익률

df_etf_ror = pd.concat([ror_1mo, ror_3mo, ror_6mo, ror_12mo])
df_etf_ror.index = df_momentum_index[1:]

str_etf_ror = str_today + str_strategy + "### ETF 1-3-6-12 개월 수익률"
display(Markdown(str_etf_ror))
display(df_etf_ror)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_ror}\n')
  df_etf_ror.to_csv(f, index=True)

### 2023-08-25 BAA_agg ### ETF 1-3-6-12 개월 수익률

,TIP,SPY,BND,QQQ,VEA,LQD,IEF,BIL,DBC,TLT,VWO
1mo_ago/2023-07-26,-1.7134,-4.0877,-1.8993,-4.3176,-6.0287,-2.6901,-2.4933,0.4529,-1.3067,-6.0219,-5.2037
3mo_ago/2023-05-24,-1.4961,6.6708,-1.1803,9.0654,-0.3281,-0.4009,-3.0330,1.3494,5.8695,-4.8215,3.0789
6mo_ago/2023-02-23,-0.1035,9.8653,-0.2330,22.0522,1.0315,0.3374,-0.9220,2.4171,1.7684,-5.7629,0.6920
12mo_ago/2022-08-23,-5.5533,7.6814,-2.6223,15.8215,9.8517,-1.7730,-5.5236,4.1881,-8.6554,-12.7144,0.4731


In [15]:
## DataFrame 만들기: 수익률 가중치
df_etf_ror_weight = df_etf_ror.copy()
df_etf_ror_weight.iloc[0] *= 12
df_etf_ror_weight.iloc[1] *= 4
df_etf_ror_weight.iloc[2] *= 2

str_etf_ror_weight = str_today + str_strategy + "### ETF 1-3-6-12 개월 가중치 적용"
display(Markdown(str_etf_ror_weight))
display(df_etf_ror_weight)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_ror_weight}\n')
  df_etf_ror_weight.to_csv(f, index=True)

### 2023-08-25 BAA_agg ### ETF 1-3-6-12 개월 가중치 적용

,TIP,SPY,BND,QQQ,VEA,LQD,IEF,BIL,DBC,TLT,VWO
1mo_ago/2023-07-26,-20.5609,-49.0527,-22.7919,-51.8118,-72.3439,-32.2809,-29.9191,5.4352,-15.6799,-72.2631,-62.4442
3mo_ago/2023-05-24,-5.9845,26.6831,-4.7213,36.2617,-1.3125,-1.6037,-12.1321,5.3977,23.4779,-19.2860,12.3157
6mo_ago/2023-02-23,-0.2071,19.7307,-0.4661,44.1045,2.0629,0.6748,-1.8440,4.8342,3.5368,-11.5258,1.3839
12mo_ago/2022-08-23,-5.5533,7.6814,-2.6223,15.8215,9.8517,-1.7730,-5.5236,4.1881,-8.6554,-12.7144,0.4731


In [16]:
df_etf_momentum_score = (12 * ror_1mo) + (4 * ror_3mo) + (2 * ror_6mo) + (ror_12mo)
df_etf_momentum_score.index = ['momentum_score']

str_etf_momentum_score = str_today + str_strategy + "### ETF 별 1-3-6-12 개월 Momentum Score"
display(Markdown(str_etf_momentum_score))
display(df_etf_momentum_score)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_momentum_score}\n')
  df_etf_momentum_score.to_csv(f, index=True)

### 2023-08-25 BAA_agg ### ETF 별 1-3-6-12 개월 Momentum Score

,TIP,SPY,BND,QQQ,VEA,LQD,IEF,BIL,DBC,TLT,VWO
momentum_score,-32.3058,5.0425,-30.6016,44.3759,-61.7419,-34.9828,-49.4188,19.8552,2.6795,-115.7892,-48.2715


In [17]:
# 12개월 이동평균 생성 및 저장
etf_12mo_ma = df_etf.rolling(window=252).mean().loc[df_etf.index[251]]  # 252 개의 영업일은 대략 12개월
df_etf_12mo_ma = pd.DataFrame([etf_12mo_ma], columns=etf_order, index=['12mo_moving_average'])

# 12개월 이동평균 수익률 생성 및 저장
df_etf_12mo_ror = (df_etf.iloc[0]/df_etf_12mo_ma - 1)*100
df_etf_12mo_ror.index = ['12mo_ma_ror(%)']

df_etf_12mo_ma = pd.concat([df_etf_12mo_ma, df_etf_12mo_ror])

str_etf_12mo_ma = str_today + str_strategy + "### ETF 별 12 개월의 Moving Average 및 수익률"
display(Markdown(str_etf_12mo_ma))
display(df_etf_12mo_ma)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_12mo_ma}\n')
  df_etf_12mo_ma.to_csv(f, index=True)

### 2023-08-25 BAA_agg ### ETF 별 12 개월의 Moving Average 및 수익률

,TIP,SPY,BND,QQQ,VEA,LQD,IEF,BIL,DBC,TLT,VWO
12mo_moving_average,106.4921,403.9233,71.6053,311.2490,43.0339,104.9623,95.9418,89.6777,24.1260,101.3168,39.5309
12mo_ma_ror(%),-0.9786,8.1616,-0.7197,16.0550,3.5927,-0.0593,-2.3157,2.3108,0.1822,-6.3235,1.8445


In [18]:
# 카나리아 자산군 0 미만 개수 세기
canary_etf_negative_count = (df_etf_momentum_score[canary_etf] < 0).sum().sum()
aggressive_portion = 0

if canary_etf_negative_count == 0:
  aggressive_portion = 100
else:
  aggressive_portion = 0


## 종목 출력하기
aggressive_etf_list_text = ''
defensive_etf_list_text = ''

# 카나리아 자산군 중 모멘텀스코어 0 미만 개수: 0
if canary_etf_negative_count == 0:
  momentum_scores_aggressive_top_1 = df_etf_momentum_score[aggressive_etf].transpose().nlargest(1, 'momentum_score').transpose()

  str_aggressive_momentum_score = str_today + str_strategy + "### 공격자산 Momentum Score"
  display(Markdown(str_aggressive_momentum_score))
  display(momentum_scores_aggressive_top_1)
  etf_columns = momentum_scores_aggressive_top_1.columns
  aggressive_etf_list_text = ' '.join([str(asset) for asset in etf_columns])

# 0 미만 카나리아 자산군 개수: 1 이상
if canary_etf_negative_count >= 1:
  ma_12mo_defensive_top_3 = df_etf_12mo_ma.iloc[1:][defensive_etf].transpose().nlargest(3, '12mo_ma_ror(%)').transpose()

  str_defensive_12mo_ma = str_today + str_strategy + "### 안전자산 Momentum Score"
  display(Markdown(str_defensive_12mo_ma))
  display(ma_12mo_defensive_top_3)

  # 안전자산 중 0 이상인 것만 필터링하기
  filtered_index = ma_12mo_defensive_top_3.columns[(ma_12mo_defensive_top_3 < 0).any()]
  df_etf_defensive_positive_index = ma_12mo_defensive_top_3.drop(filtered_index, axis=1).columns
  defensive_etf_list_text = ' '.join([str(asset) for asset in df_etf_defensive_positive_index])

str_canary_text = "## 카나리아 자산군 중 모멘텀스코어 0 미만 개수: " + str(canary_etf_negative_count)
str_aggressive_text = "## 공격자산 비율: " + str(aggressive_portion) + " %"
display(Markdown(str_today[1:] + str_strategy))
display(Markdown(str_canary_text))
display(Markdown(str_aggressive_text))

str_aggressive_etf_list = '## 추천 공격자산(각 100%): '
if canary_etf_negative_count == 0:
  str_aggressive_etf_list += aggressive_etf_list_text

str_defensive_etf_list = '## 추천 안전자산(각 33.3%): '
if canary_etf_negative_count >= 1:
  str_defensive_etf_list += defensive_etf_list_text

str_defensive_etf_count = '## 안전자산 수익률 0 이상 개수'

display(Markdown(str_aggressive_etf_list))
display(Markdown(str_defensive_etf_list))

# 안전자산 개수 및 비율
str_etf_defensive_positive_index = '## 안전자산 투자 개수: ' + str(len(df_etf_defensive_positive_index))
display(Markdown(str_etf_defensive_positive_index))
str_etf_defensive_positive_portion = '## 안전자산 투자 비율: ' + str(round(len(df_etf_defensive_positive_index) * 1/3 * 100, 1)) + " %"
display(Markdown(str_etf_defensive_positive_portion))

# csv에 자산 투자 조언 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_today[1:] + str_strategy}\n')
  f.write(f'{str_canary_text}\n')
  f.write(f'{str_aggressive_text}\n')
  f.write(f'{str_aggressive_etf_list}\n')
  f.write(f'{str_defensive_etf_list}\n')
  f.write(f'{str_etf_defensive_positive_index}\n')
  f.write(f'{str_etf_defensive_positive_portion}')

### 2023-08-25 BAA_agg ### 안전자산 Momentum Score

,BIL,DBC,LQD
12mo_ma_ror(%),2.3108,0.1822,-0.0593


## 2023-08-25 BAA_agg 

## 카나리아 자산군 중 모멘텀스코어 0 미만 개수: 3

## 공격자산 비율: 0 %

## 추천 공격자산(각 100%): 

## 추천 안전자산(각 33.3%): BIL DBC

## 안전자산 투자 개수: 2

## 안전자산 투자 비율: 66.7 %